In [19]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [50]:
faces = pd.read_csv("faces.utf8.csv")
briefs = pd.read_csv("briefs.utf8.csv")
dict_col = {}
dict_col["25_49csp"] = 12
dict_col["cspp"] = 15
dict_col["urb"] = 18
dict_col["all"] = 14
dict_col["f25_49rda"] = 17
dict_col["f25_49csp"] = 16
dict_col["15_24"] = 10
dict_col["25_49"] = 11
dict_col["25_49urb"] = 13
cpm = pd.read_csv("cpm.utf8.csv")
#cpm.sort_values('cpm', ascending=False, inplace=True)
tmp = np.array([6.319625, 2.083290, 4.578867, 2.754223, 1, 8.543064, 10.461262, 9.185119, 1.330685])
#cpm

In [57]:
faces_cpm = np.matrix(faces.iloc[:, 10:19], dtype=float)
cpm_matrix = np.eye(9, dtype=float)
cpm_row = tmp
cpm_tomult = cpm_row.T * cpm_matrix
faces_cpm *= cpm_tomult
max_cpm = faces_cpm.max(1)
print(max_cpm)

[[  52233.081166]
 [  58509.20803 ]
 [  50967.268464]
 ..., 
 [ 156358.280737]
 [ 150924.626874]
 [ 132157.122846]]


In [58]:
faces['rev_max'] = max_cpm
print(faces)

                   reference  code_postal      nom_commune  \
0       01043.00003.02.01.01         1700          BEYNOST   
1       01043.00003.02.02.01         1700          BEYNOST   
2       01049.00001.01.01.01         1120        LA BOISSE   
3       01049.00001.01.02.01         1120        LA BOISSE   
4       01053.00003.02.01.01         1000  BOURG-EN-BRESSE   
5       01053.00003.02.02.01         1000  BOURG-EN-BRESSE   
6       01053.00004.01.01.01         1000  BOURG-EN-BRESSE   
7       01053.00005.01.01.01         1000  BOURG-EN-BRESSE   
8       01053.00008.01.01.01         1000  BOURG-EN-BRESSE   
9       01053.00012.01.01.01         1000  BOURG-EN-BRESSE   
10      01053.00012.03.01.01         1000  BOURG-EN-BRESSE   
11      01053.00016.03.01.01         1000  BOURG-EN-BRESSE   
12      01053.00016.03.02.01         1000  BOURG-EN-BRESSE   
13      01053.00020.01.01.01         1000  BOURG-EN-BRESSE   
14      01053.00021.02.01.01         1000  BOURG-EN-BRESSE   
15      

In [22]:
briefs

,week,ordre,cible,contacts
0,1,1,25_49csp,87200000
1,1,2,cspp,53200000
2,1,3,urb,464600000
3,1,4,25_49csp,152300000
4,1,5,cspp,50300000
5,1,6,f25_49rda,6500000
6,1,7,cspp,21300000
7,1,8,25_49csp,153800000
8,1,9,all,440400000
9,1,10,all,54000000


In [23]:
from ortools.linear_solver import pywraplp

briefs_data = briefs.values
faces_array = faces_data.values
panneaux = [[]] * len(faces_array)
solver = pywraplp.Solver('SolveOptimization', pywraplp.Solver.GLOP_LINEAR_PROGRAMMING)

objective = solver.Objective()
for i in range(0, len(faces_array)):
    panneaux[i] = solver.NumVar(0, 1, faces_array[i][0])
    objective.SetCoefficient(panneaux[i], 1)
objective.SetMinimization()

constraints = [0] * 1
constraints[0] = solver.Constraint(briefs_data[0][3], 140000000)
for j in range (0, len(faces_array)):
    constraints[0].SetCoefficient(panneaux[j], faces_array[j][5])

status = solver.Solve()

if status == solver.OPTIMAL:
    print('yes')
    nb_panneaux = 0
    for i in range (0, len(faces_array)):
        if (panneaux[i].solution_value() > 0):
            nb_panneaux += 1
            print(panneaux[i])
    print(nb_panneaux)
else:
    if status == solver.FEASIBLE:
        print('not opti')
    else:
        print('not solve')

NameError: name 'faces_data' is not defined